In [61]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

## File paths

In [14]:
train_dataset_path = "sign-language-mnist/sign_mnist_train/sign_mnist_train.csv"
test_dataset_path =  "sign-language-mnist/sign_mnist_test/sign_mnist_test.csv"
train_dataset = pd.read_csv(train_dataset_path)
test_dataset = pd.read_csv(test_dataset_path)


## Load train and test files

In [53]:
def load_dataset():
    train_set_x_orig = np.array(train_dataset.iloc[:,1:]) # train set features
    train_set_y_orig = np.array(train_dataset.iloc[:,:1]) # train set labels
    test_set_x_orig = np.array(test_dataset.iloc[:,1:]) # test set features
    test_set_y_orig = np.array(test_dataset.iloc[:,:1]) # test set labels
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig

In [62]:
# Function to convert to one hot encoding
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

In [73]:
# Flatten the data
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig = load_dataset()
X_train = X_train_orig/255.
X_test = X_test_orig/255.
Y_train = convert_to_one_hot(Y_train_orig, 25)
Y_test = convert_to_one_hot(Y_test_orig, 25)

In [74]:
print ("number of training examples = " + str(X_train.shape[1]))
print ("number of test examples = " + str(X_test.shape[1]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 784
number of test examples = 784
X_train shape: (27455, 784)
Y_train shape: (25, 27455)
X_test shape: (7172, 784)
Y_test shape: (25, 7172)
